In [1]:
import pandas as pd
import numpy 

In [2]:
f13 = pd.read_pickle('./raw_data/f13.pkl')
price = pd.read_pickle('./raw_data/price.pkl')

In [3]:
# data processing
f13 = f13.drop_duplicates()
f13_new = f13.loc[(f13['iQTY'] != 0) & (f13['iMARKET_VALUE'] != 0) & (f13['iFILING_DATE'] > '2013-06-30')]
f13_new

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iAMEND,iRESTATEMENT,iTYPE,iQTY,iMARKET_VALUE,iLONG_FRACTION
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,0,0,0,96510.0,3366000.0,0.023443
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,0,0,0,70214.0,5249000.0,0.036557
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,0,0,0,346115.0,15471000.0,0.107748
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,0,0,0,882340.0,11347000.0,0.079026
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,0,0,0,129065.0,3620000.0,0.025212
...,...,...,...,...,...,...,...,...,...,...,...
57569231,74561590,1730479,039483102,2020-03-31,2020-04-23,0,0,0,300.0,11000.0,0.000086
57569232,74561591,1730479,88160R101,2020-03-31,2020-04-23,0,0,0,25.0,13000.0,0.000102
57569233,74561592,1730479,921937793,2020-03-31,2020-04-23,0,0,0,230.0,25000.0,0.000196
57569234,74561593,1730479,863667101,2020-03-31,2020-04-23,0,0,0,1030.0,171000.0,0.001341


In [85]:
# The fund must complete filings within 46 days of the quarter-end.
f13_new['iPERIOD_END'] = pd.to_datetime(f13_new['iPERIOD_END'], format = '%Y-%m-%d')
f13_new['iFILING_DATE'] = pd.to_datetime(f13_new['iFILING_DATE'], format = '%Y-%m-%d')
f13_new['filing_gap_day'] = (f13_new['iFILING_DATE'] - f13_new['iPERIOD_END']).apply(lambda x: x.days)
f13_new = f13_new.loc[f13_new['filing_gap_day'] <= 46]


# The fund must be valued between $100M and $500M 
tmp_value = f13_new.groupby(['iCIK', 'iPERIOD_END'])['iMARKET_VALUE'].sum().reset_index()
invalid_value = tmp_value[(tmp_value['iMARKET_VALUE'] > 5e8) | (tmp_value['iMARKET_VALUE'] < 1e8)][['iCIK', 'iPERIOD_END']]
# for i, row in invalid_value.iterrows():       # very very slow... [??]
#     f13_new = f13_new.drop((f13_new['iCIK'] == row.iCIK) & ((f13_new['iPERIOD_END'] == row.iPERIOD_END)))


# The fund must have exactly one filing point for each quarter  [verified]
tmp = f13_new.groupby(['iCIK', 'iPERIOD_END', 'iFILING_DATE'])['filing_gap_day'].unique().reset_index()
tmp['filing_gap_day_unique'] = tmp['filing_gap_day'].apply(lambda x: len(x) == 1)       
tmp['filing_gap_day_unique'].all()    # True


# The fund must have between 20 and 200 holdings (each quarter??)
tmp_holding = f13_new.groupby(['iCIK', 'iPERIOD_END'])['iCUSIP'].count().reset_index()
invalid_holding = tmp_holding[(tmp_holding['iCUSIP'] > 200) | (tmp_holding['iCUSIP'] < 20)][['iCIK', 'iPERIOD_END']]
# for i, row in invalid_holding.iterrows():       # very very slow... [??]
#     f13_new = f13_new.drop((f13_new['iCIK'] == row.iCIK) & ((f13_new['iPERIOD_END'] == row.iPERIOD_END)))

27339400


In [70]:
1e10

10000000000.0